<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebooks" data-toc-modified-id="Notebooks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notebooks</a></span><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Mark-false-positive-notebooks-and-notebooks-with-broken-format" data-toc-modified-id="Mark-false-positive-notebooks-and-notebooks-with-broken-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Mark false-positive notebooks and notebooks with broken format</a></span></li><li><span><a href="#Mark-empty-notebooks" data-toc-modified-id="Mark-empty-notebooks-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Mark empty notebooks</a></span></li><li><span><a href="#Mark-fork-duplicates" data-toc-modified-id="Mark-fork-duplicates-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Mark fork duplicates</a></span></li><li><span><a href="#Mark-duplicates" data-toc-modified-id="Mark-duplicates-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Mark duplicates</a></span></li><li><span><a href="#Mark-restricted-toy" data-toc-modified-id="Mark-restricted-toy-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Mark restricted toy</a></span></li><li><span><a href="#Mark-toy-examples" data-toc-modified-id="Mark-toy-examples-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Mark toy examples</a></span></li></ul></li></ul></div>

# Notebooks

Analyze notebooks: programming languages, python version, number of cells by notebookk, and notebook names.

In [1]:
import sys

sys.path.insert(0, "../archaeology")

import numpy as np
import pandas as pd

from db import connect, Query


%matplotlib inline

## Load

In [2]:
with connect() as session:
    print("Last query:", session.query(Query).all()[-1])
    notebooks = pd.read_sql_table("notebooks", session.connection())

Last query: <Query(language:"Jupyter Notebook" created:2018-04-16T03:56:05Z..2018-04-16T22:18:58Z)>


In [3]:
len(notebooks)

1450071

In [4]:
notebooks["skip"] = 0

## Mark false-positive notebooks and notebooks with broken format

In [5]:
mask = ~(
    ~((notebooks["kernel"] == "no-kernel") & (notebooks["nbformat"] == "0"))
    & (notebooks["total_cells"] != 0)
)
SKIP = 1024
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

14698

## Mark empty notebooks

In [6]:
mask = notebooks["total_cells"] == notebooks["empty_cells"]
SKIP = 512
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

26395

## Mark fork duplicates

In [7]:
mask = notebooks.duplicated(subset=["name", "sha1_source"])
SKIP = 256
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

137181

## Mark duplicates

In [8]:
mask = notebooks.duplicated(subset=["sha1_source"])
SKIP = 128
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

290312

## Mark restricted toy

In [9]:
mask = (notebooks["homework_count"] != 0) | (
    (notebooks["course_count"] != 0)
    & (
        (notebooks["assignment_count"] != 0)
        | (notebooks["lesson_count"] != 0)
        | (notebooks["exercise_count"] != 0)
    )
)
SKIP = 64
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

106619

## Mark toy examples

In [10]:
mask = (
    (notebooks["homework_count"] != 0)
    | (notebooks["course_count"] != 0)
    | (notebooks["lesson_count"] != 0)
    | (notebooks["assignment_count"] != 0)
    | (notebooks["exercise_count"] != 0)
)
SKIP = 32
notebooks.loc[mask, "skip"] = notebooks[mask]["skip"] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

357589

In [11]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebooks
    set skip = 0;
    """
    )

CPU times: user 11.7 ms, sys: 463 µs, total: 12.1 ms
Wall time: 1min 4s


In [12]:
import tqdm

with_skip = notebooks[notebooks["skip"] != 0]

with connect() as session:
    for id_, skip in tqdm.tqdm(
        zip(with_skip["id"], with_skip["skip"]), total=len(with_skip)
    ):
        # print(id_)
        session.execute(
            "UPDATE notebooks SET skip = {} WHERE id = {};".format(skip, id_)
        )
    session.commit()

100%|██████████| 543493/543493 [04:27<00:00, 2030.53it/s]


In [13]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebook_features AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 10.5 ms, sys: 1.15 ms, total: 11.6 ms
Wall time: 36.8 s


In [14]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebook_modules AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 22 ms, sys: 1.73 ms, total: 23.7 ms
Wall time: 1min 25s


In [15]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebook_asts AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 19.4 ms, sys: 8.47 ms, total: 27.9 ms
Wall time: 2min 20s


In [16]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebook_names AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 23.4 ms, sys: 5 ms, total: 28.4 ms
Wall time: 3min 51s


In [17]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE notebook_markdowns AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 23.6 ms, sys: 0 ns, total: 23.6 ms
Wall time: 1min 26s


In [18]:
%%time
with connect() as session:
    session.execute(
        """
    UPDATE executions AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """
    )
    session.commit()

CPU times: user 21.3 ms, sys: 6.82 ms, total: 28.1 ms
Wall time: 3min 29s
